In [1]:
import numpy as np
import tensorflow as tf
import tensornets as nets

from tensornets.datasets import voc
data_dir = '/home/taehoonlee/Data/VOCdevkit/VOC2007'
data_name = 'test'

In [2]:
with tf.Graph().as_default():
    inputs = tf.placeholder(tf.float32, [None, 412, 412, 3])
    models = []
    with tf.device('gpu:0'):
        models += [
            nets.YOLOv2(inputs, nets.Darknet19),
            nets.YOLOv2VOC(inputs)
        ]
    with tf.device('gpu:1'):
        models += [
            nets.TinyYOLOv2(inputs, nets.TinyDarknet19),
            nets.TinyYOLOv2VOC(inputs)
        ]
    imgs = voc.load(data_dir, data_name, total_num=10)
    results = [[] for _ in range(len(models))]

    with tf.Session() as sess:
        nets.pretrained(models)
        for (img, scale) in imgs:
            outs = sess.run(models, {inputs: models[1].preprocess(img)})
            print("%s %.8f %.8f" % (
                img.shape[1:3],
                np.sum((outs[0] - outs[1]) ** 2),
                np.sum((outs[2] - outs[3]) ** 2)))
            for i in range(len(models)):
                results[i].append(models[i].get_boxes(outs[i], img.shape[1:3]))

(500, 353) 0.00000000 0.00000004
(500, 335) 0.00000000 0.00000000
(375, 500) 0.00000000 0.00000000
(406, 500) 0.00000000 0.00000000
(375, 500) 0.00000000 0.00000000
(375, 500) 0.00000000 0.00000000
(480, 354) 0.00000000 0.00000000
(324, 500) 0.00000000 0.00000000
(375, 500) 0.00000000 0.00000000
(333, 500) 0.00000000 0.00000000


In [3]:
with tf.Graph().as_default():
    inputs = tf.placeholder(tf.float32, [None, None, None, 3])
    models = []
    with tf.device('gpu:0'):
        models += [
            nets.FasterRCNN(inputs, nets.VGG16, stem_out='conv5/3'),
            nets.FasterRCNN_VGG16_VOC(inputs)
        ]
    with tf.device('gpu:1'):
        models += [
            nets.FasterRCNN(inputs, nets.ZF),
            nets.FasterRCNN_ZF_VOC(inputs)
        ]
    imgs = voc.load(data_dir, data_name, min_shorter_side=600, total_num=10)
    results = [[] for _ in range(len(models))]

    with tf.Session() as sess:
        nets.pretrained(models)
        for (img, scale) in imgs:
            outs = sess.run(models, {inputs: models[0].preprocess(img),
                                     models[0].scales: scale,
                                     models[1].scales: scale,
                                     models[2].scales: scale,
                                     models[3].scales: scale})
            print("%s %.8f %.8f" % (
                img.shape[1:3],
                np.sum((outs[0] - outs[1]) ** 2),
                np.sum((outs[2] - outs[3]) ** 2)))
            for i in range(len(models)):
                results[i].append(models[i].get_boxes(outs[i], img.shape[1:3]))

(850, 600) 0.00000000 0.00000000
(896, 600) 0.00000000 0.00000000
(600, 800) 0.00000000 0.00000000
(600, 739) 0.00000030 0.00000000
(600, 800) 0.00000000 0.00000000
(600, 800) 0.00000000 0.00000000
(814, 600) 0.00000000 0.00000000
(600, 926) 0.00000000 0.00000000
(600, 800) 0.00000000 0.00000000
(600, 901) 0.00000000 0.00000000
